In [ ]:
!pip install autogluon

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.0/138.0 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 12.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.5/259.5 kB 24.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of openxlab to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of openxlab to determine which version is compatible with other requirem

In [ ]:
import pandas as pd
import numpy as np
from autogluon.tabular import TabularPredictor
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd

directory = '/content/drive/My Drive/DataMining/ieee-fraud-detection/'

#directory = '/content/IEEEfraud/'  # directory where you have downloaded the data CSV files from the competition
label = 'isFraud'  # name of target variable to predict in this competition
eval_metric = 'roc_auc'  # Optional: specify that competition evaluation metric is AUC
save_path = directory + 'AutoGluonModels/'  # where to store trained models
train_identity = pd.read_csv(directory+'train_identity.csv')
train_transaction = pd.read_csv(directory+'train_transaction.csv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
train_data = pd.merge(train_transaction, train_identity, on='TransactionID', how='left')

In [ ]:
predictor = TabularPredictor(label=label, eval_metric=eval_metric, path=save_path, verbosity=3).fit(
    train_data, presets='good_quality', time_limit=3600, excluded_model_types=['NN', 'STACKER'],keep_only_best=True
)

results = predictor.fit_summary()

Verbosity: 3 (Detailed Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.10.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Thu Jun 27 21:05:47 UTC 2024
CPU Count:          12
GPU Count:          1
Memory Avail:       62.74 GB / 83.48 GB (75.2%)
Disk Space Avail:   10.31 GB / 15.00 GB (68.7%)
Presets specified: ['good_quality']
============ fit kwarg info ============
User Specified kwargs:
{'auto_stack': True,
 'excluded_model_types': ['NN', 'STACKER'],
 'keep_only_best': True,
 'num_bag_sets': 1,
 'refit_full': True,
 'save_bag_folds': False,
 'set_best_to_refit_full': True}
Full kwargs:
{'_feature_generator_kwargs': None,
 '_save_bag_folds': None,
 'ag_args': None,
 'ag_args_ensemble': None,
 'ag_args_fit': None,
 'auto_stack': True,
 'calibrate': 'auto',
 'ds_args': {'clean_up_fits': True,
             'detection_time_frac': 0.25,
             'enable_ray_logging': True,
  

*** Summary of fit() ***
Estimated performance of each model:
                      model score_val eval_metric pred_time_val    fit_time pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0  WeightedEnsemble_L3_FULL      None     roc_auc          None  393.991227                   None          11.143680            3       True          5
1      LightGBM_BAG_L2_FULL      None     roc_auc          None  303.802216                   None          21.004536            2       True          4
2      LightGBM_BAG_L1_FULL      None     roc_auc          None   42.320791                   None          42.320791            1       True          2
3    LightGBMXT_BAG_L2_FULL      None     roc_auc          None  361.843011                   None          79.045331            2       True          3
4    LightGBMXT_BAG_L1_FULL      None     roc_auc          None  240.476889                   None         240.476889            1       True          1
Number of models tra

In [ ]:
# For the training data
print(train_data.columns)

# For the test data
print(test_data.columns)


Index(['TransactionID', 'isFraud', 'TransactionDT', 'TransactionAmt',
       'ProductCD', 'card1', 'card2', 'card3', 'card4', 'card5',
       ...
       'id_31', 'id_32', 'id_33', 'id_34', 'id_35', 'id_36', 'id_37', 'id_38',
       'DeviceType', 'DeviceInfo'],
      dtype='object', length=434)
Index(['TransactionID', 'TransactionDT', 'TransactionAmt', 'ProductCD',
       'card1', 'card2', 'card3', 'card4', 'card5', 'card6',
       ...
       'id_31', 'id_32', 'id_33', 'id_34', 'id_35', 'id_36', 'id_37', 'id_38',
       'DeviceType', 'DeviceInfo'],
      dtype='object', length=433)


In [ ]:
# Rename columns with hyphens in the test data to match the training data
test_data.columns = test_data.columns.str.replace('-', '_')


In [ ]:
missing_in_test = set(test_identity.columns) - set(test_data.columns)
missing_in_test

set()

In [ ]:
#test_transaction = pd.read_csv(directory+'test_transaction.csv')
missing_in_test_trans = set(test_identity.columns) - set(test_transaction.columns)
missing_in_test_trans




{'DeviceInfo',
 'DeviceType',
 'id-01',
 'id-02',
 'id-03',
 'id-04',
 'id-05',
 'id-06',
 'id-07',
 'id-08',
 'id-09',
 'id-10',
 'id-11',
 'id-12',
 'id-13',
 'id-14',
 'id-15',
 'id-16',
 'id-17',
 'id-18',
 'id-19',
 'id-20',
 'id-21',
 'id-22',
 'id-23',
 'id-24',
 'id-25',
 'id-26',
 'id-27',
 'id-28',
 'id-29',
 'id-30',
 'id-31',
 'id-32',
 'id-33',
 'id-34',
 'id-35',
 'id-36',
 'id-37',
 'id-38'}

In [ ]:

# For the test data
print(test_identity.columns)


Index(['TransactionID', 'id-01', 'id-02', 'id-03', 'id-04', 'id-05', 'id-06',
       'id-07', 'id-08', 'id-09', 'id-10', 'id-11', 'id-12', 'id-13', 'id-14',
       'id-15', 'id-16', 'id-17', 'id-18', 'id-19', 'id-20', 'id-21', 'id-22',
       'id-23', 'id-24', 'id-25', 'id-26', 'id-27', 'id-28', 'id-29', 'id-30',
       'id-31', 'id-32', 'id-33', 'id-34', 'id-35', 'id-36', 'id-37', 'id-38',
       'DeviceType', 'DeviceInfo'],
      dtype='object')


In [ ]:
test_identity.columns = test_identity.columns.str.replace('-', '_')


In [ ]:
test_identity = pd.read_csv(directory+'test_identity.csv')
test_transaction = pd.read_csv(directory+'test_transaction.csv')

# Replace hyphens with underscores in test_identity column names to match training data
test_identity.columns = test_identity.columns.str.replace('-', '_')

# Merge the dataframes using an outer join to ensure all columns are present
test_data = pd.merge(test_transaction, test_identity, on='TransactionID', how='outer')  # outer join to include all columns

# Fill NaN values with a suitable strategy, such as -999
test_data = test_data.fillna(-999)  # Or use another appropriate strategy

y_predproba = predictor.predict_proba(test_data)
y_predproba.head(5)

Loading: /content/drive/My Drive/DataMining/ieee-fraud-detection/AutoGluonModels/models/LightGBMXT_BAG_L1_FULL/model.pkl
Loading: /content/drive/My Drive/DataMining/ieee-fraud-detection/AutoGluonModels/models/LightGBM_BAG_L1_FULL/model.pkl
Loading: /content/drive/My Drive/DataMining/ieee-fraud-detection/AutoGluonModels/models/LightGBMXT_BAG_L2_FULL/model.pkl
Loading: /content/drive/My Drive/DataMining/ieee-fraud-detection/AutoGluonModels/models/LightGBM_BAG_L2_FULL/model.pkl
Loading: /content/drive/My Drive/DataMining/ieee-fraud-detection/AutoGluonModels/models/WeightedEnsemble_L3_FULL/model.pkl


,0,1
0,0.794174,0.205826
1,0.973957,0.026043
2,0.997900,0.002100
3,0.782334,0.217666
4,0.986708,0.013292


In [ ]:
predictor.positive_class


1

In [ ]:
predictor.class_labels  # classes in this list correspond to columns of predict_proba() output

[0, 1]

In [ ]:
y_predproba = predictor.predict_proba(test_data, as_multiclass=False)

Loading: /content/drive/My Drive/DataMining/ieee-fraud-detection/AutoGluonModels/models/LightGBMXT_BAG_L1_FULL/model.pkl
Loading: /content/drive/My Drive/DataMining/ieee-fraud-detection/AutoGluonModels/models/LightGBM_BAG_L1_FULL/model.pkl
Loading: /content/drive/My Drive/DataMining/ieee-fraud-detection/AutoGluonModels/models/LightGBMXT_BAG_L2_FULL/model.pkl
Loading: /content/drive/My Drive/DataMining/ieee-fraud-detection/AutoGluonModels/models/LightGBM_BAG_L2_FULL/model.pkl
Loading: /content/drive/My Drive/DataMining/ieee-fraud-detection/AutoGluonModels/models/WeightedEnsemble_L3_FULL/model.pkl


In [ ]:
submission = pd.read_csv(directory+'sample_submission.csv')
submission['isFraud'] = y_predproba
submission.head()
submission.to_csv(directory+'my_submission.csv', index=False)